# ANN_1

# ANN_1

## `Step 1` : Import

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import  TensorDataset, DataLoader

## `Step 2` : Create DataLoader

In [27]:
# 데이터 불러오기
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target

# 데이터 분할
y = df['label']
X = df.drop(['label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values,
                                                   random_state=42, stratify=y)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)



## `Step 3` : Set Network Structure

In [28]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(4, 16)
        self.hidden_layer1 = nn.Linear(16, 32)
        self.output_layer = nn.Linear(32, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        out = self.output_layer(out)
        return out

## `Step 4` : Create Model instance

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device = {device}')
model = NeuralNetwork().to(device)


device = cuda

## `Step 5` : Model compile

In [30]:
# 모델 컴파일
learning_rate = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## `Step 6` : Set train loop

In [34]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)

    for batch, (X,y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)

        # loss
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch * len(X)
        print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')

## `Step 7` : Set test loop

In [36]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f'Test Error : \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n')

## `Step 8` : Run model

In [37]:
# 모델 실행
epochs = 10

for i in range(epochs):
    print(f'Epoch {i+1} \n------------------------------')
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)

print("Done!")

Epoch 1 
------------------------------
loss: 1.104175  [    0]/  112
loss: 1.063668  [   10]/  112
loss: 1.001417  [   20]/  112
loss: 0.974181  [   30]/  112
loss: 1.099196  [   40]/  112
loss: 1.131157  [   50]/  112
loss: 1.029493  [   60]/  112
loss: 1.046669  [   70]/  112
loss: 0.991781  [   80]/  112
loss: 0.980483  [   90]/  112
loss: 0.995154  [  100]/  112
loss: 1.178336  [   22]/  112
Test Error : 
 Accuracy: 34.2%, Avg loss: 1.013189

Epoch 2 
------------------------------
loss: 0.998811  [    0]/  112
loss: 1.072228  [   10]/  112
loss: 0.861224  [   20]/  112
loss: 1.145309  [   30]/  112
loss: 0.923511  [   40]/  112
loss: 1.029394  [   50]/  112
loss: 1.038562  [   60]/  112
loss: 0.923659  [   70]/  112
loss: 1.022684  [   80]/  112
loss: 0.942879  [   90]/  112
loss: 1.011915  [  100]/  112
loss: 0.944836  [   22]/  112
Test Error : 
 Accuracy: 34.2%, Avg loss: 0.978484

Epoch 3 
------------------------------
loss: 0.973278  [    0]/  112
loss: 0.857192  [   10]/  